In [1]:
import numpy as np
import scipy
import pandas as pd
import math
import random
import sklearn
from nltk.corpus import stopwords
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from scipy.sparse.linalg import svds
from sklearn.preprocessing import MinMaxScaler
from elasticsearch import Elasticsearch
import pandas as pd
from pandasticsearch import Select

#import matplotlib.pyplot as plt

In [2]:
es = Elasticsearch('http://3.125.9.240:9221')
result_dict = es.search(index="profiles22", body={"query": {"match_all": {}}})
articles_df = pd.DataFrame(result_dict['hits']['hits'])


/tmp/ipykernel_424/1192434381.py:2: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  result_dict = es.search(index="profiles22", body={"query": {"match_all": {}}})


In [48]:

# PATH_CustData = "../data/customerData"
# 
# articles_df = pd.read_excel(f'{PATH_CustData}/customer_behavioral_data_with_city_v5.xlsx',index_col=[0]) 

articles_df['store_city_name'] =pd.DataFrame(articles_df._source.values.tolist())['store_city_name'].fillna('NA')

products_df = articles_df[['product_id','product_name']]

articles_df.head(5)


KeyError: "None of [Index(['product_id', 'product_name'], dtype='object')] are in the [columns]"

In [4]:
interactions_df = articles_df[['customer_id','product_id','product_name','Event','store_city_name']]
interactions_df.head(10)

,customer_id,product_id,product_name,Event,store_city_name
0,11627839,13690016,بلاك فانتوم,cart,الرياض
1,9384042,11495803,G-10,cart,الرياض
2,12923822,39961752,قماش جوخ S501-2 3D,cart,NA
3,3288761,40249527,دهن الشيوخ دهن الشيخ عبدالله الجهني,cart,جدة
4,1385607,34462642,مجموعة(50)صحن بلاستيك مستطيل ابيض رقم نص,cart,NA
5,13346796,38326571,حلاوة برميل توكسيك سويتس حامض سوبر ساور منوعة ...,cart,مكة
6,13349742,38747399,شرائط الفراولة 200غ - Strawberry Belt,cart,NA
7,10815148,6358398,عطر 222 - بازل لانتيردي,cart,الرياض
8,8461453,29902541,مبخرة شعر الكترونية لاسلكية,cart,جدة
9,7017438,37854546,رسيفر beIN HD عربي مع اشتراك 6 شهور في باقة ت...,cart,الرياض


In [5]:
event_type_strength = {
   'rating': 1.0,
   'wishlist': 2.5, 
   'cart': 3.5, 
   'purchased': 4.5, 
}

In [6]:
interactions_df['eventStrength'] = interactions_df['Event'].apply(lambda x: event_type_strength[x])

interactions_df.head(5)

/tmp/ipykernel_11621/282239076.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interactions_df['eventStrength'] = interactions_df['Event'].apply(lambda x: event_type_strength[x])


,customer_id,product_id,product_name,Event,store_city_name,eventStrength
0,11627839,13690016,بلاك فانتوم,cart,الرياض,3.5
1,9384042,11495803,G-10,cart,الرياض,3.5
2,12923822,39961752,قماش جوخ S501-2 3D,cart,NA,3.5
3,3288761,40249527,دهن الشيوخ دهن الشيخ عبدالله الجهني,cart,جدة,3.5
4,1385607,34462642,مجموعة(50)صحن بلاستيك مستطيل ابيض رقم نص,cart,NA,3.5


In [7]:
users_interactions_count_df = interactions_df.groupby(['customer_id', 'product_id','product_name']).size().groupby('customer_id').size()
print('# users: %d' % len(users_interactions_count_df))
users_with_enough_interactions_df = users_interactions_count_df[users_interactions_count_df >= 3].reset_index()[['customer_id']]
print('# users with at least 3 interactions: %d' % len(users_with_enough_interactions_df))

# users: 222410
# users with at least 3 interactions: 24557


In [8]:
print('# of interactions: %d' % len(interactions_df))
interactions_from_selected_users_df = interactions_df.merge(users_with_enough_interactions_df, 
               how = 'right',
               left_on = 'customer_id',
               right_on = 'customer_id')
print('# of interactions from users with at least 2 interactions: %d' % len(interactions_from_selected_users_df))
interactions_from_selected_users_df.sample(2)

# of interactions: 560000
# of interactions from users with at least 2 interactions: 166540


,customer_id,product_id,product_name,Event,store_city_name,eventStrength
159698,12319998,18894861,باليت ظل عيون من جلانز,wishlist,الرياض,2.5
90868,4429555,3118056,ماسكارا فولس لاش اكس فايبر من لوريال - اسود,rating,الرياض,1.0


In [9]:
def smooth_user_preference(x):
    return math.log(1+x, 2)
    
interactions_full_df = interactions_from_selected_users_df \
                    .groupby(['customer_id', 'product_id','product_name','store_city_name'])['eventStrength'].sum() \
                    .apply(smooth_user_preference).reset_index()
print('# of unique user/item interactions: %d' % len(interactions_full_df))
print('# of Unique Customers: %d ' % interactions_full_df.customer_id.nunique())
interactions_full_df.head(10)

# of unique user/item interactions: 111558
# of Unique Customers: 24557 


,customer_id,product_id,product_name,store_city_name,eventStrength
0,258,4027132,لومينوس,الرياض,1.807355
1,258,6349726,توباكو,الرياض,1.807355
2,258,22774629,مخدة رويال ميد || Royal mid pillow,بريدة,1.807355
3,299,4828807,عود مروكي محسن - افضل انواع العود المروكي، 32 ...,الرياض,1.584963
4,299,10444964,عطر كمستري - Chemistry عطر الأوركيد الفريد، 50 مل,الرياض,1.584963
5,299,31951474,عباية سوداء بقصة جاكيت 1174,القطيف,1.807355
6,387,1443808,روج سحري /1قطعة,المدينة المنورة,1.000000
7,387,4258776,شبشب نسائي,جدة,2.459432
8,387,20513708,لعبة اطفال,المدينة المنورة,1.000000
9,387,32881699,مقص كبير,المدينة المنورة,1.000000


In [10]:
interactions_train_df, interactions_test_df = train_test_split(interactions_full_df,
                                   stratify=interactions_full_df['customer_id'], 
                                   test_size=0.23,
                                   random_state=42)

print('# interactions on Train set: %d' % len(interactions_train_df))
print('# interactions on Test set: %d' % len(interactions_test_df))
interactions_full_df.head(5)

# interactions on Train set: 85899
# interactions on Test set: 25659


,customer_id,product_id,product_name,store_city_name,eventStrength
0,258,4027132,لومينوس,الرياض,1.807355
1,258,6349726,توباكو,الرياض,1.807355
2,258,22774629,مخدة رويال ميد || Royal mid pillow,بريدة,1.807355
3,299,4828807,عود مروكي محسن - افضل انواع العود المروكي، 32 ...,الرياض,1.584963
4,299,10444964,عطر كمستري - Chemistry عطر الأوركيد الفريد، 50 مل,الرياض,1.584963


In [11]:
#Indexing by personId to speed up the searches during evaluation
interactions_full_indexed_df = interactions_full_df.set_index('customer_id')
interactions_train_indexed_df = interactions_train_df.set_index('customer_id')
interactions_test_indexed_df = interactions_test_df.set_index('customer_id')

In [12]:
def get_items_interacted(person_id, interactions_df):
    # Get the user's data and merge in the movie information.
    interacted_items = interactions_df.loc[person_id]['product_id']
    return set(interacted_items if type(interacted_items) == pd.Series else [interacted_items])

# **Popularity Model - Overall and CityWise**

In [13]:
#Computes the most popular items
print("\t CityWise Popular Items")
item_popularity_df_city = interactions_full_df.groupby(['product_id','product_name','store_city_name'])['eventStrength'].sum().sort_values(ascending=False).reset_index()
item_popularity_df_city.to_csv("../data/customerData/item_popularity_city.csv",index=False)
item_popularity_df_city.sample(5)


	 CityWise Popular Items


,product_id,product_name,store_city_name,eventStrength
3368,3370636,جبن سائل كريم نادك 2 حبة 500 جرام,بريدة,8.813781
14201,40064999,قالب سيليكون للشمعة,المدينة المنورة,2.169925
3735,18817922,عينة ايف سان لوران ليبر انتنس او دو بارفيوم,الرياض,7.924813
21302,27285757,شاي أخضر نعناع العطار,جدة,1.000000
6032,7884342,بصل يمني طازج 1كجم,NA,4.629357


In [14]:
print("\t OverAll Popular Items")
item_popularity_df = interactions_full_df[['product_id','product_name','eventStrength']]
item_popularity_df = interactions_full_df.groupby(['product_id','product_name'])['eventStrength'].sum().sort_values(ascending=False).reset_index()
item_popularity_df.head(10)

	 OverAll Popular Items


,product_id,product_name,eventStrength
0,6657244,عباية بسحاب مخفي وجيوب,1510.592833
1,22435617,عباية كسرات من الخلف,1458.416549
2,2515568,عباية عمليه كلوش,1429.347955
3,20804714,طرحة ربط سادة 4 ألوان,1397.086860
4,6370334,عطر 247 - بازل سوفاج,1371.132884
5,3688124,عطر 128 - بازل ديلينا,1226.135241
6,5348474,نقاب بلوقو,1129.409595
7,5348507,نقاب واسع بلوقو,1083.585784
8,7125012,عباية بسحاب اسود وكم مزموم,1008.234693
9,11901827,عطر 273 - بازل لوران ليبر,1004.884759


In [15]:
class PopularityRecommender:
    
    MODEL_NAME = 'Popularity'
    
    def __init__(self, popularity_df, items_df=None):
        self.popularity_df = popularity_df
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        # Recommend the more popular items that the user hasn't seen yet.
        recommendations_df = self.popularity_df[~self.popularity_df['product_id'].isin(items_to_ignore)] \
                               .sort_values('eventStrength', ascending = False) \
                               .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'product_id', 
                                                          right_on = 'product_id').head(topn)[['eventStrength', 'product_id', 'product_name']]


        return recommendations_df
    
popularity_model = PopularityRecommender(item_popularity_df, articles_df)

In [16]:
print(" \t Recommend Popular products for the Customer # : " , 1778457)
popularity_model.recommend_items(1778457, topn=15).merge(articles_df[['product_id','product_name']], how = 'inner', 
                                                          left_on = ['product_id','product_name'], 
                                                          right_on = ['product_id','product_name']).drop_duplicates()



 	 Recommend Popular products for the Customer # :  1778457


,product_id,product_name,eventStrength
0,6657244,عباية بسحاب مخفي وجيوب,1510.592833
3000,22435617,عباية كسرات من الخلف,1458.416549
6379,2515568,عباية عمليه كلوش,1429.347955
9498,20804714,طرحة ربط سادة 4 ألوان,1397.086860
12460,6370334,عطر 247 - بازل سوفاج,1371.132884
14850,3688124,عطر 128 - بازل ديلينا,1226.135241
16785,5348474,نقاب بلوقو,1129.409595
18807,5348507,نقاب واسع بلوقو,1083.585784
20670,7125012,عباية بسحاب اسود وكم مزموم,1008.234693
22223,11901827,عطر 273 - بازل لوران ليبر,1004.884759


# **Content-Based Filtering model**

In [17]:
!pip install tashaphyne # for stemming arabic words

In [18]:
import nltk
nltk.download('stopwords')

#@title Text Processing
nltk.download('stopwords')
nltk.download('punkt')
random.seed(2022)

arabic_stop_words = '''
،
ء
ءَ
آ
آب
آذار
آض
آل
آمينَ
آناء
آنفا
آه
آهاً
آهٍ
آهِ
أ
أبدا
أبريل
أبو
أبٌ
أجل
أجمع
أحد
أخبر
أخذ
أخو
أخٌ
أربع
أربعاء
أربعة
أربعمئة
أربعمائة
أرى
أسكن
أصبح
أصلا
أضحى
أطعم
أعطى
أعلم
أغسطس
أفريل
أفعل به
أفٍّ
أقبل
أكتوبر
أل
ألا
ألف
ألفى
أم
أما
أمام
أمامك
أمامكَ
أمد
أمس
أمسى
أمّا
أن
أنا
أنبأ
أنت
أنتم
أنتما
أنتن
أنتِ
أنشأ
أنه
أنًّ
أنّى
أهلا
أو
أوت
أوشك
أول
أولئك
أولاء
أولالك
أوّهْ
أى
أي
أيا
أيار
أيضا
أيلول
أين
أيّ
أيّان
أُفٍّ
ؤ
إحدى
إذ
إذا
إذاً
إذما
إذن
إزاء
إلى
إلي
إليكم
إليكما
إليكنّ
إليكَ
إلَيْكَ
إلّا
إمّا
إن
إنَّ
إى
إياك
إياكم
إياكما
إياكن
إيانا
إياه
إياها
إياهم
إياهما
إياهن
إياي
إيهٍ
ئ
ا
ا?
ا?ى
االا
االتى
ابتدأ
ابين
اتخذ
اثر
اثنا
اثنان
اثني
اثنين
اجل
احد
اخرى
اخلولق
اذا
اربعة
اربعون
اربعين
ارتدّ
استحال
اصبح
اضحى
اطار
اعادة
اعلنت
اف
اكثر
اكد
الآن
الألاء
الألى
الا
الاخيرة
الان
الاول
الاولى
التى
التي
الثاني
الثانية
الحالي
الذاتي
الذى
الذي
الذين
السابق
الف
اللاتي
اللتان
اللتيا
اللتين
اللذان
اللذين
اللواتي
الماضي
المقبل
الوقت
الى
الي
اليه
اليها
اليوم
اما
امام
امس
امسى
ان
انبرى
انقلب
انه
انها
او
اول
اي
ايار
ايام
ايضا
ب
بؤسا
بإن
بئس
باء
بات
باسم
بان
بخٍ
بد
بدلا
برس
بسبب
بسّ
بشكل
بضع
بطآن
بعد
بعدا
بعض
بغتة
بل
بلى
بن
به
بها
بهذا
بيد
بين
بَسْ
بَلْهَ
ة
ت
تاء
تارة
تاسع
تانِ
تانِك
تبدّل
تجاه
تحت
تحوّل
تخذ
ترك
تسع
تسعة
تسعمئة
تسعمائة
تسعون
تسعين
تشرين
تعسا
تعلَّم
تفعلان
تفعلون
تفعلين
تكون
تلقاء
تلك
تم
تموز
تينك
تَيْنِ
تِه
تِي
ث
ثاء
ثالث
ثامن
ثان
ثاني
ثلاث
ثلاثاء
ثلاثة
ثلاثمئة
ثلاثمائة
ثلاثون
ثلاثين
ثم
ثمان
ثمانمئة
ثمانون
ثماني
ثمانية
ثمانين
ثمنمئة
ثمَّ
ثمّ
ثمّة
ج
جانفي
جدا
جعل
جلل
جمعة
جميع
جنيه
جوان
جويلية
جير
جيم
ح
حاء
حادي
حار
حاشا
حاليا
حاي
حبذا
حبيب
حتى
حجا
حدَث
حرى
حزيران
حسب
حقا
حمدا
حمو
حمٌ
حوالى
حول
حيث
حيثما
حين
حيَّ
حَذارِ
خ
خاء
خاصة
خال
خامس
خبَّر
خلا
خلافا
خلال
خلف
خمس
خمسة
خمسمئة
خمسمائة
خمسون
خمسين
خميس
د
دال
درهم
درى
دواليك
دولار
دون
دونك
ديسمبر
دينار
ذ
ذا
ذات
ذاك
ذال
ذانك
ذانِ
ذلك
ذهب
ذو
ذيت
ذينك
ذَيْنِ
ذِه
ذِي
ر
رأى
راء
رابع
راح
رجع
رزق
رويدك
ريال
ريث
رُبَّ
ز
زاي
زعم
زود
زيارة
س
ساء
سابع
سادس
سبت
سبتمبر
سبحان
سبع
سبعة
سبعمئة
سبعمائة
سبعون
سبعين
ست
ستة
ستكون
ستمئة
ستمائة
ستون
ستين
سحقا
سرا
سرعان
سقى
سمعا
سنة
سنتيم
سنوات
سوف
سوى
سين
ش
شباط
شبه
شتانَ
شخصا
شرع
شمال
شيكل
شين
شَتَّانَ
ص
صاد
صار
صباح
صبر
صبرا
صدقا
صراحة
صفر
صهٍ
صهْ
ض
ضاد
ضحوة
ضد
ضمن
ط
طاء
طاق
طالما
طرا
طفق
طَق
ظ
ظاء
ظل
ظلّ
ظنَّ
ع
عاد
عاشر
عام
عاما
عامة
عجبا
عدا
عدة
عدد
عدم
عدَّ
عسى
عشر
عشرة
عشرون
عشرين
عل
علق
علم
على
علي
عليك
عليه
عليها
علًّ
عن
عند
عندما
عنه
عنها
عوض
عيانا
عين
عَدَسْ
غ
غادر
غالبا
غدا
غداة
غير
غين
ـ
ف
فإن
فاء
فان
فانه
فبراير
فرادى
فضلا
فقد
فقط
فكان
فلان
فلس
فهو
فو
فوق
فى
في
فيفري
فيه
فيها
ق
قاطبة
قاف
قال
قام
قبل
قد
قرش
قطّ
قلما
قوة
ك
كأن
كأنّ
كأيّ
كأيّن
كاد
كاف
كان
كانت
كانون
كثيرا
كذا
كذلك
كرب
كسا
كل
كلتا
كلم
كلَّا
كلّما
كم
كما
كن
كى
كيت
كيف
كيفما
كِخ
ل
لأن
لا
لا سيما
لات
لازال
لاسيما
لام
لايزال
لبيك
لدن
لدى
لدي
لذلك
لعل
لعلَّ
لعمر
لقاء
لكن
لكنه
لكنَّ
للامم
لم
لما
لمّا
لن
له
لها
لهذا
لهم
لو
لوكالة
لولا
لوما
ليت
ليرة
ليس
ليسب
م
مئة
مئتان
ما
ما أفعله
ما انفك
ما برح
مائة
ماانفك
مابرح
مادام
ماذا
مارس
مازال
مافتئ
ماي
مايزال
مايو
متى
مثل
مذ
مرّة
مساء
مع
معاذ
معه
معها
مقابل
مكانكم
مكانكما
مكانكنّ
مكانَك
مليار
مليم
مليون
مما
من
منذ
منه
منها
مه
مهما
ميم
ن
نا
نبَّا
نحن
نحو
نعم
نفس
نفسه
نهاية
نوفمبر
نون
نيسان
نيف
نَخْ
نَّ
ه
هؤلاء
ها
هاء
هاكَ
هبّ
هذا
هذه
هل
هللة
هلم
هلّا
هم
هما
همزة
هن
هنا
هناك
هنالك
هو
هي
هيا
هيهات
هيّا
هَؤلاء
هَاتانِ
هَاتَيْنِ
هَاتِه
هَاتِي
هَجْ
هَذا
هَذانِ
هَذَيْنِ
هَذِه
هَذِي
هَيْهات
و
و6
وأبو
وأن
وا
واحد
واضاف
واضافت
واكد
والتي
والذي
وان
واهاً
واو
واوضح
وبين
وثي
وجد
وراءَك
ورد
وعلى
وفي
وقال
وقالت
وقد
وقف
وكان
وكانت
ولا
ولايزال
ولكن
ولم
وله
وليس
ومع
ومن
وهب
وهذا
وهو
وهي
وَيْ
وُشْكَانَ
ى
ي
ياء
يفعلان
يفعلون
يكون
يلي
يمكن
يمين
ين
يناير
يوان
يورو
يوليو
يوم
يونيو
ّأيّان

'''

[nltk_data] Downloading package stopwords to /home/user2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package stopwords to /home/user2/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/user2/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#@title Sample Data after cleaning text
import re
import string
import sys
import argparse
from nltk.tokenize import word_tokenize
from tashaphyne.stemming import ArabicLightStemmer

arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

'''
The first step is to subject the data to preprocessing.
This involves removing both arabic and english punctuation
Normalizing different letter variants with one common letter
'''
stopwords = nltk.corpus.stopwords.words('english')
# Arabic stop words with nltk
stop_words = stopwords



arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

arabic_diacritics = re.compile("""
                             ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)

ArListem = ArabicLightStemmer()

def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ؤ", "ء", text)
    text = re.sub("ئ", "ء", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    
    return text


def remove_diacritics(text):
    text = re.sub(arabic_diacritics, '', text)
    return text


def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)


def remove_repeating_char(text):
    return re.sub(r'([a-zA-Z])\1+', r'\1', text)

def remove_stop_words(text):
    tokens = word_tokenize(text)
    text = ' '.join(word for word in tokens if word not in stop_words)
    return text

def removing_arabic_stopwords(text):
    words = word_tokenize(text) # it works on one hadith not list
    stemmed_words = " ".join([word for word in words if word not in arabic_stop_words])    
    return stemmed_words

def stemming(text):
    
    words = word_tokenize(text) 
    stemed_text = " ".join([ArListem.light_stem(word) if (ArListem.light_stem(word) not in arabic_stop_words) else word for word in words])
    return stemed_text

def clean_text(text):
    '''
        Make text lowercase, remove text in square brackets,remove links,remove punctuation
        and remove words containing numbers.
    '''

    bannedWords ={ "مل","جرام","ml","gram","gm","nbsp","DESCRIPTION","amp"}
    re_banned_words = re.compile(r"\b(" + "|".join(bannedWords) + ")\\W", re.I)
    text = re_banned_words.sub(" ", text)
    #print(re_banned_words)


    text = re.sub(r'\d', ' ', text)
    text = re.sub('https?://\S+|www\.\S+', '', text)                # remove urls
    text = re.sub('<.*?>+', '', text)                               # remove tages
    text = re.sub(r'@[^\s]+', ' ', text)                            # Removing @user
    text = re.sub(r'#([^\s]+)', r'\1', text)                        # remove #word with word
    text = re.sub('[%s]' % re.escape(punctuations_list), '', text)  # remove punctuation
    text = re.sub(r'\b\w{1,2}\b', '', text)                         # remove token with length <=2
    text = re.sub('\n', '', text)                                   # remove new line
    text = re.sub(r'\s+', ' ', text)                                # Removing multiple spaces


    return text

def preprocess_data(text):
    
    text = removing_arabic_stopwords(text)
    text = remove_diacritics(text)
    text = remove_stop_words(text)
    text = normalize_arabic(text)
    text = clean_text(text)
    text = remove_punctuations(text)
    text = stemming(text)
   
    return text

articles_df['product_description'] = articles_df['product_description'].fillna('')
articles_df['cleaned_text'] = articles_df['product_name'].apply(preprocess_data) #+ " " + articles_df['product_description'].apply(preprocess_data)


In [ ]:
#Ignoring stopwords (words with no semantics) from English and Arabic (as we have a corpus with mixed languages)
#stopwords_list = stopwords.words('english') + stopwords.words('arabic')

#Trains a model whose vectors size is 5000, composed by the main unigrams and bigrams found in the corpus, ignoring stopwords
vectorizer = TfidfVectorizer(analyzer = 'word',
                                       min_df= 0.0001,
                                       max_df = 0.95,
                                       strip_accents = None,
                                       encoding = 'utf-8', 
                                       preprocessor=None,
                                       token_pattern=r'(?u)\b\w*[^\d+\s*(a|g|ml|جرام|غ|مل)]\w*\b',
                                       max_features=5000,
                                       lowercase=True)

item_ids = articles_df['product_id'].tolist()
tfidf_matrix = vectorizer.fit_transform(articles_df['cleaned_text']) #articles_df['product_name'] + "" + articles_df['product_description'])
tfidf_feature_names = vectorizer.get_feature_names()
tfidf_matrix

In [ ]:
tfidf_view = pd.DataFrame(tfidf_matrix.toarray(), columns = vectorizer.get_feature_names())
tfidf_view

In [ ]:
def get_item_profile(item_id):

    idx = item_ids.index(item_id)
    item_profile = tfidf_matrix[idx:idx+1]
    return item_profile

def get_item_profiles(ids):

    item_profiles_list = [get_item_profile(x) for x in ids]
    item_profiles = scipy.sparse.vstack(item_profiles_list)
    return item_profiles

def build_users_profile(person_id, interactions_indexed_df):
    interactions_person_df = interactions_indexed_df.loc[[person_id]]
    #print("person_id : ",person_id, "\n")
    user_item_profiles = get_item_profiles(interactions_person_df['product_id'])
    
    user_item_strengths = np.array(interactions_person_df['eventStrength']).reshape(-1,1)
    #Weighted average of item profiles by the interactions strength
    user_item_strengths_weighted_avg = np.sum(user_item_profiles.multiply(user_item_strengths), axis=0) / np.sum(user_item_strengths)
    user_profile_norm = sklearn.preprocessing.normalize(user_item_strengths_weighted_avg)
    return user_profile_norm

def build_users_profiles(): 
    interactions_indexed_df = interactions_train_df[interactions_train_df['product_id'] \
                                                   .isin(articles_df['product_id'])].set_index('customer_id')
    user_profiles = {}
    for person_id in interactions_indexed_df.index.unique():
        user_profiles[person_id] = build_users_profile(person_id, interactions_indexed_df)
    return user_profiles

In [ ]:
user_profiles = build_users_profiles()
len(user_profiles)

# **Check a User Profile**


In [ ]:
uid = 11862461 #1778457,10324509,3567476
myprofile = user_profiles[uid]

ud = pd.DataFrame(sorted(zip(tfidf_feature_names, 
                        user_profiles[uid].flatten().tolist()), key=lambda x: -x[1])[:50],
             columns=['token', 'relevance'])


print( "User Profile, Top 15 keywords :", uid)
ud[ud.relevance > 0][:15]


In [ ]:
class ContentBasedRecommender:
    
    MODEL_NAME = 'Content-Based'
    
    def __init__(self, items_df=None):
        self.item_ids = item_ids
        self.items_df = items_df
        
    def get_model_name(self):
        return self.MODEL_NAME
        
    def _get_similar_items_to_user_profile(self, person_id, topn=3000):
        #Computes the cosine similarity between the user profile and all item profiles
        cosine_similarities = cosine_similarity(user_profiles[person_id], tfidf_matrix)
        #Gets the top similar items
        similar_indices = cosine_similarities.argsort().flatten()[-topn:]
        #Sort the similar items by similarity
        similar_items = sorted([(item_ids[i], cosine_similarities[0,i]) for i in similar_indices], key=lambda x: -x[1])
        return similar_items
        
    def recommend_items(self, user_id, items_to_ignore=[], topn=10, verbose=False):
        similar_items = self._get_similar_items_to_user_profile(user_id)
        #Ignores items the user has already interacted
        similar_items_filtered = list(filter(lambda x: x[0] not in items_to_ignore, similar_items))
        
        recommendations_df = pd.DataFrame(similar_items_filtered, columns=['product_id', 'recStrength']) \
                                    .head(topn)

        if verbose:
            if self.items_df is None:
                raise Exception('"items_df" is required in verbose mode')

            recommendations_df = recommendations_df.merge(self.items_df, how = 'left', 
                                                          left_on = 'product_id', 
                                                          right_on = 'product_id')[['recStrength', 'product_id', 'product_name']]


        return recommendations_df
    
content_based_recommender_model = ContentBasedRecommender(articles_df)

In [ ]:
print("\tProducts Recommendations for User # :" % uid)
contentRecomDf = content_based_recommender_model.recommend_items(uid, topn=300).merge(articles_df[['product_id','product_name']], how = 'inner', 
                                                          left_on = 'product_id', 
                                                          right_on = 'product_id').drop_duplicates()[['product_name', 'recStrength', 'product_id']]
contentRecomDf.sample(25)

